<a href="https://colab.research.google.com/github/esmeee51/B134030044/blob/main/HW2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

**目標**

本專案旨在透過紅酒的13項化學成分特徵，建立分類模型以預測紅酒所屬的品種，包含三類：Class_0、Class_1、Class_2。

**任務類型**

多類別監督式分類（Multiclass Supervised Classification）。

**使用模型**

K-Nearest Neighbors（KNN）演算法

**評估指標**
- 主要：Accuracy（準確率）
- 輔助：Precision、Recall、F1-score
- 使用 Confusion Matrix（混淆矩陣）觀察各類別預測表現

## 資料收集

| 來源                   | 特徵數 | 樣本數 | 類別 |
| -------------------- | --- | --- | -- |
| scikit-learn 內建 Wine | 13  | 178 | 3  |

特徵欄位：

- alcohol
- malic_acid
- ash
- alcalinity_of_ash
- magnesium
- total_phenols
- flavanoids
- nonflavanoid_phenols
- proanthocyanins
- color_intensity
- hue
- od280/od315_of_diluted_wines
- proline

In [ ]:
from sklearn.datasets import load_wine
wine = load_wine()
X, y = wine.data, wine.target
feature_names, target_names = wine.feature_names, wine.target_names

## 資料前處理

### 資料清理

載入資料後檢查缺失值，結果顯示無缺漏。

In [ ]:
import pandas as pd

df = pd.DataFrame(X, columns=feature_names).assign(target=y)
assert not df.isna().any().any()

Wine 資料集無缺失值。

### 探索性分析

透過 seaborn.pairplot 視覺化變數分布與類別區分性。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(data=df, x="alcohol", y="color_intensity", hue="target")
plt.show()

部分特徵（如 proline、color_intensity）對類別分離效果良好，部分則重疊明顯。

### 資料分割

訓練測試比為 8:2，使用 stratify 保持類別分布一致。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

### 特徵縮放

由於 KNN 仰賴距離計算，特徵必須標準化。


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 模型訓練

採用 KNN 演算法訓練模型，使用 KNN，k=5，距離為歐氏距離。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

knn_clf = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=5, weights="uniform", metric="euclidean")
)

knn_clf.fit(X_train, y_train)

## 模型評估

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = knn_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

## 模型調整

### 調參目標

透過 GridSearchCV 交叉驗證調整參數

In [ ]:
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())
param_grid = {
    "kneighborsclassifier__n_neighbors": range(3, 16),
    "kneighborsclassifier__weights": ["uniform", "distance"],
    "kneighborsclassifier__metric": ["euclidean", "manhattan"]
}

grid = GridSearchCV(pipe, param_grid, cv=10, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

print("最佳參數：", grid.best_params_)
print(f"CV 平均 Accuracy：{grid.best_score_:.3f}")

best_model = grid.best_estimator_

### 測試集驗證最佳模型

In [ ]:
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy (Best Model): {acc:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

## 模型部署

### 儲存模型

In [ ]:
import joblib
joblib.dump({
    "pipeline": best_model,
    "target_names": target_names
}, "wine_knn_pipeline.joblib")

### 推論預測

In [ ]:
import numpy as np
artifacts = joblib.load("wine_knn_pipeline.joblib")
pipe, names = artifacts["pipeline"], artifacts["target_names"]

sample = [[13.0, 2.1, 2.3, 16.0, 101.0, 2.5, 2.3, 0.3, 1.7, 5.0, 1.0, 3.0, 1000.0]]
pred_idx = pipe.predict(sample)[0]
print("預測類別：", names[pred_idx])

## 結論

本專案成功利用 KNN 演算法對 Wine 資料集進行分類，透過標準化與 GridSearch 調參，在測試集上達到高準確率。由於 Wine 資料為中型規模、具良好特徵分佈，KNN 能有效發揮距離判斷優勢。

**優點**
- 不須訓練時間，部署快速
- 實作簡單、解釋直觀
- 效果穩定，適用於結構良好資料集

**建議與延伸**
- 若資料特徵維度更多，建議加入 PCA 等降維技術
- 可與 SVM、Random Forest 等模型比較效能